# 인프런 수강평 스크랩핑

## Selenium 및 웹 드라이버 설치

In [109]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


## 라이브러리 import

In [110]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('-disable-dev-shm-usage')

In [140]:
def collect_review(reviews, wd) : 
    global review_idx  # 전역 변수로 선언
    for review in reviews:
        author = review.find_element(By.CLASS_NAME,'author').text.split(": ")[1]
        course = review.find_element(By.CLASS_NAME,'relative_course').text.split(": ")[1]
        content = review.find_element(By.CSS_SELECTOR, 'div.post_content > span').text
        course_url = review.find_element(By.CSS_SELECTOR,'div.post_content div.post_metas > a').get_attribute('href')
        datetime = review.find_element(By.CLASS_NAME,'updated_at').get_attribute("datetime")
        
        star_solid = wd.find_element(By.CLASS_NAME,"star_solid").get_attribute("style")  # "width: 100%"
        rating = int(star_solid.split(":")[1][:-2])/20  # 100 /20 = 5점

        

        print(f" 수강평 #{review_idx+1}:", [course, rating, content, author, datetime,course_url])
        review_df.loc[review_idx] = [course, rating, content, author, datetime,course_url]
        review_idx += 1
    
    return review_df


## reivew_scraping() : 리뷰 스크래핑 함수

In [141]:
review_idx = 0

def reivew_scraping(url, wd):
    global review_idx  # 전역 변수로 선언
    review_df = pd.DataFrame(columns=("Course","Rating","Content","Author","DateTime", "CourseURL"))

    reviews = wd.find_elements(By.CLASS_NAME, 'media-content')
    review_df = collect_review(reviews, wd)
    try:
        for i in range(1000):  
            wd.find_element(By.CLASS_NAME, 'pagination-next').click()
            reviews = wd.find_elements(By.CLASS_NAME, 'media-content')
            print("다음페이지")
            review_df.append = collect_review(reviews)
            time.sleep(1)
    
    except: 
        pass

    return review_df

## scraping() : 스크래핑 함수

In [142]:
def scraping():
    wd = webdriver.Chrome('chromedriver', options = chrome_options)
    wd.implicitly_wait(3)

    url = 'https://www.inflearn.com/community/reviews'
    wd.get(url)

    review_df = reivew_scraping(url, wd)

    wd.close()

    return review_df

In [143]:
review_df = scraping()

 수강평 #1: ['ElasticSearch Essential', 5.0, '동작원리 잘 배웠습니다', 'sujeong_lee4', '2023-04-11T06:00:24.000Z', 'https://www.inflearn.com/course/elasticsearch-essential']
 수강평 #2: ['개발 초보를 위한 구체적인 프론트엔드 로드맵 다잇소', 5.0, '강의 잘 들어보았습니다. 섬세하게 처음부터 빠짐없이 취업준비하는 방법을 알려주셔서 도움이 많이 됐습니다. 백엔드부터 프론트까지 로드맵 전자책을 구매했습니다. 로드맵이 구체적이여서 이대로만 따라하면 충분히 중고신입정도 수준으로 회사에서 높게 평가받을 수 있을 것 같습니다. 미리 알았다면 뜬구름 잡지 않고 더 시간을 단축할 수 있었을 것 같아서 지금 알게 되어서 아쉽다는 생각도 듭니다. 부트캠퍼스에 대한 강사님의 평가도 판단이 안서던 제게 강의 선택에 큰 가이드라인이 되어주어 나은 선택을 할 수 있을 것 같습니다. 앞으로도 저런 개발 서비스에 대한 비교 및 평가에 대한 내용이 있으면 초보자로서 판단이 잘 안설 때 판단의 근거로 삼을 수 있을것 같습니다. 각 챕터마다 사용법이 조금 더 자세했으면 좋겠다는 생각은 했으나 전반적으로 준비해야 할 사항의 큰 틀을 잡아주셔서 등대처럼 지침대로 잘 따라하면 될 것 같다고 생각했습니다. 감사합니다.', '장윤경', '2023-04-11T05:58:11.000Z', 'https://www.inflearn.com/course/%EA%B0%9C%EB%B0%9C-%EC%B4%88%EB%B3%B4-%ED%94%84%EB%A1%A0%ED%8A%B8%EC%97%94%EB%93%9C-%EB%A1%9C%EB%93%9C%EB%A7%B5-%EB%8B%A4%EC%9E%87%EC%86%8C']
 수강평 #3: ['스프링 핵심 원리 - 기본편', 5.0, '서버 개발을 하고있지만 뭔가 내가 근본적으로 잘 알고 있는게 맞는지,,, 이런 감정을 느끼는 공허한 중니어 서버 개발자에게 정말 

In [135]:
review_df

,Course,Rating,Content,Author,DateTime,CourseURL
0,ElasticSearch Essential,5.0,동작원리 잘 배웠습니다,sujeong_lee4,2023-04-11T06:00:24.000Z,https://www.inflearn.com/course/elasticsearch-...
1,개발 초보를 위한 구체적인 프론트엔드 로드맵 다잇소,5.0,강의 잘 들어보았습니다. 섬세하게 처음부터 빠짐없이 취업준비하는 방법을 알려주셔서 ...,장윤경,2023-04-11T05:58:11.000Z,https://www.inflearn.com/course/%EA%B0%9C%EB%B...
2,스프링 핵심 원리 - 기본편,5.0,"서버 개발을 하고있지만 뭔가 내가 근본적으로 잘 알고 있는게 맞는지,,, 이런 감정...",skywhite15,2023-04-11T05:53:07.000Z,https://www.inflearn.com/course/%EC%8A%A4%ED%9...
3,[하루 10분|Web Project] HTML/JS/CSS로 나만의 심리테스트 사이...,5.0,지루했던 js수업이 mbti와 유형검사 이런 흥미로운 주제로 진행하니까 강의가 지루...,shaaa6256,2023-04-11T05:21:52.000Z,https://www.inflearn.com/course/%EC%8B%AC%EB%A...
4,스프링부트 개념정리(이론),5.0,강의는 너무 훌륭합니다. 다만 아쉬운 것은 관련되어 이어질 수 있는 강의가 없다는 ...,gaabi1204,2023-04-11T05:16:42.000Z,https://www.inflearn.com/course/%EC%8A%A4%ED%9...
5,코딩테스트 [ ALL IN ONE ],5.0,최근 트렌드에는 좀 뒤떨어진느낌,wisehero,2023-04-11T05:13:52.000Z,https://www.inflearn.com/course/%EC%BD%94%EB%9...
6,실습으로 끝장내는 파이썬 웹 크롤링과 웹 페이지 자동화,5.0,물흐르는듯한 설명으로 정말 잘배웠습니다. 특히 이런건 이렇게 하면 됩니다 식의 설명...,김퀴즈,2023-04-11T05:08:31.000Z,https://www.inflearn.com/course/%EC%8B%A4%EC%8...
7,스프링 핵심 원리 - 기본편,5.0,선생님 강의는 항상 최고 에요 b 잘들었습니다,김정훈,2023-04-11T04:59:27.000Z,https://www.inflearn.com/course/%EC%8A%A4%ED%9...
8,도메인 주도 설계(DDD) 기반 마이크로서비스(MSA) 모델링,5.0,좋은 강의 잘 들었습니다. 덕분에 많이 배웠습니다.,스타빈,2023-04-11T04:52:31.000Z,https://www.inflearn.com/course/%EB%8F%84%EB%A...
9,스프링 핵심 원리 - 기본편,5.0,강의 구매하기 전에 수강생 분들 평점을 보고 신기하면서 궁금했지만 강의를 수강하는 ...,민석,2023-04-11T04:12:54.000Z,https://www.inflearn.com/course/%EC%8A%A4%ED%9...


In [132]:
#test
review_idx = 0

def reivew_scraping(url, wd):
    global review_idx  # 전역 변수로 선언
    review_df = pd.DataFrame(columns=("Course","Rating","Content","Author","DateTime", "CourseURL"))

    reviews = wd.find_elements(By.CLASS_NAME, 'media-content')
    for review in reviews:
        author = review.find_element(By.CLASS_NAME,'author').text.split(": ")[1]
        course = review.find_element(By.CLASS_NAME,'relative_course').text.split(": ")[1]
        content = review.find_element(By.CSS_SELECTOR, 'div.post_content > span').text
        course_url = review.find_element(By.CSS_SELECTOR,'div.post_content div.post_metas > a').get_attribute('href')
        datetime = review.find_element(By.CLASS_NAME,'updated_at').get_attribute("datetime")
        
        star_solid = wd.find_element(By.CLASS_NAME,"star_solid").get_attribute("style")  # "width: 100%"
        rating = int(star_solid.split(":")[1][:-2])/20  # 100 /20 = 5점

        

        print(f" 수강평 #{review_idx+1}:", [course, rating, content, author, datetime,course_url])
        review_df.loc[review_idx] = [course, rating, content, author, datetime,course_url]
        review_idx += 1

    return review_df